## Prepare wav

In [16]:
def write_lines(lines, file):
    with open(file, 'w') as f:
        for line in lines:
            f.write("%s\n" % line.replace("\n", ""))

In [31]:
import glob
import os

def generate_csv(base_dir, sentences_file, output_file):
    with open(sentences_file) as f:
        sentences = f.readlines()
    dirs = os.listdir(base_dir)
    dirs.sort(key=int)
    trans_all = []
    for dir in dirs:
        files = list(map(lambda x: x.replace(".wav",""), os.listdir(base_dir + "/" + dir)))
        files.sort(key=int)
        for file in files:
            index = int(file.replace(".wav",""))
            if (index < 0 or index >= len(sentences)):
                raise Exception("File does not have a corresponding transcription. Should be between {} and {}".format(0, len(sentences)))
            trans_all.append("{},{}/{}/{}.wav".format(sentences[index],base_dir,dir,file))
    write_lines(trans_all, output_file)
generate_csv("MelaNetDataIVR/data_ivr/wav", "MelaNetDataIVR/data_ivr/sentences", "MelaNetDataIVR/data_ivr/trans.csv")

## Prepare json files describing train and valid data 

In [33]:
from src.json_generator import generate_json
from random import shuffle

def split_file(file):
    lines = open(file).readlines()
    shuffle(lines)
    train_end = int(len(lines)*.95)
    train = lines[:train_end]
    valid = lines[train_end:]
    return train,valid

train,valid = split_file("MelaNetDataIVR/data_ivr/trans.csv")
write_lines(train, "MelaNetDataIVR/data_ivr/train_trans.csv")
write_lines(valid, "MelaNetDataIVR/data_ivr/valid_trans.csv")
generate_json("MelaNetDataIVR/data_ivr/train_trans.csv", ".", "MelaNetDataIVR/data_ivr/train_corpus.json")
generate_json("MelaNetDataIVR/data_ivr/valid_trans.csv", ".", "MelaNetDataIVR/data_ivr/valid_corpus.json")

## Remove characters with the same sound

In [36]:
from src.json_generator import generate_json
from src.redundant import remove_redundant
supported = "ሁ ለ ም ራ ሮ ሰ ስ ሶ ባ ት ን ኝ አ ዎ ዘ ዜ ይ ደ ድ ጠ".split()

remove_redundant("MelaNetDataIVR/data_ivr/train_corpus.json", char_set=supported)
remove_redundant("MelaNetDataIVR/data_ivr/valid_corpus.json", char_set=supported)

== Unsupported characters ==
set()
== Character stat ==
[(' ', 8550), ('ት', 5533), ('አ', 4005), ('ስ', 3971), ('ም', 2362), ('ለ', 1631), ('ድ', 1626), ('ን', 1563), ('ዎ', 841), ('ሁ', 832), ('ደ', 799), ('ይ', 799), ('ሰ', 786), ('ባ', 786), ('ዜ', 785), ('ሮ', 785), ('ራ', 775), ('ጠ', 747), ('ኝ', 747), ('ዘ', 747), ('ሶ', 746)]
== Unsupported characters ==
set()
== Character stat ==
[(' ', 450), ('ት', 307), ('ስ', 221), ('አ', 187), ('ም', 112), ('ለ', 89), ('ድ', 86), ('ን', 71), ('ጠ', 55), ('ዘ', 55), ('ኝ', 55), ('ሁ', 54), ('ሶ', 46), ('ሰ', 42), ('ባ', 42), ('ራ', 39), ('ደ', 35), ('ይ', 35), ('ዜ', 33), ('ሮ', 33), ('ዎ', 33)]
